# Глубокая нейронная сеть для классификации изображений: на основе Pytorch

In [1]:
import torch
import torch.nn as nn
import numpy as np
from dnn_app_utils_v3 import load_data
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Hyper-parameters 
input_size = 12288     # num_px * num_px * 3
hidden_size  = 7
output_size  = 1
num_epochs = 1000
learning_rate = 0.0075

## 1. Данные

In [3]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
print(train_x_orig.shape, train_y.shape, test_x_orig.shape, test_y.shape)

train_x = torch.from_numpy(train_x_orig.reshape(-1, 64 * 64 * 3)).type(torch.float32) / 255.
test_x = torch.from_numpy(test_x_orig.reshape(-1, 64 * 64 * 3)).type(torch.float32) / 255.
train_y = torch.from_numpy(train_y.reshape(-1, 1)).type(torch.float32)
test_y = torch.from_numpy(test_y.reshape(-1, 1)).type(torch.float32)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(209, 64, 64, 3) (1, 209) (50, 64, 64, 3) (1, 50)
torch.Size([209, 12288]) torch.Size([209, 1]) torch.Size([50, 12288]) torch.Size([50, 1])


In [4]:
train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

## 2. Нейронная сеть

In [5]:
model = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, output_size), nn.Sigmoid())
model

Sequential(
  (0): Linear(in_features=12288, out_features=7, bias=True)
  (1): ReLU()
  (2): Linear(in_features=7, out_features=1, bias=True)
  (3): Sigmoid()
)

## 3. Оптмизатор и лосс-функция

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 4. Обучение

In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):    
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [100/1000], Loss: 0.0544
Epoch [200/1000], Loss: 0.0255
Epoch [300/1000], Loss: 0.0031
Epoch [400/1000], Loss: 0.0118
Epoch [500/1000], Loss: 0.0018
Epoch [600/1000], Loss: 0.0013
Epoch [700/1000], Loss: 0.0008
Epoch [800/1000], Loss: 0.0005
Epoch [900/1000], Loss: 0.0017
Epoch [1000/1000], Loss: 0.0015


In [8]:
 with torch.no_grad():  
    correct = 0
    total = 0
    for images, labels in train_loader:
        outputs = model(images)
        predicted, _ = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.round() == labels.ravel()).sum()

    print('Accuracy of the model on the train images: {} %'.format(100 * correct / total))

Accuracy of the model on the train images: 100.0 %


In [9]:
 with torch.no_grad():  
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        predicted, _ = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.round() == labels.ravel()).sum()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 66.0 %
